# Competition 2 - Image Segmentation
The competition 2 is about image segmentation. We need to use Tensorflow 2.0 (with keras) in order to perform semantic segmentation of only 2 different classes inside satellite captures: "roof" and "no roof". We tried different approaches and we will present the 2 main final ones (the output of this notebook, however, will be not the same we had in the challenge for computer performance reasons).

## From Scratch
The first part is the code used to model a U-Net based neural network without transfer learning. 
***
The final best result comes from 4 different training sessions on the same model, changing optimizer, learning rate, validation split and other main parameters; this is done in order to let the network to learn in steps, giving it more data at every step and changing the "speed" of the learning process.

In [1]:
# ------------------------------------------------IMPORTS AND DIRECTORIES DEFINITIONS

from IPython.core.interactiveshell import InteractiveShell
from IPython.display import FileLink, FileLinks

InteractiveShell.ast_node_interactivity = "all"

import os, random, time, sys, scipy.misc
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from PIL import Image

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
from keras.layers import BatchNormalization as BatchNorm
from keras.models import Model, load_model
from keras.layers import Input, Reshape, Activation
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from keras import layers


get_ipython().run_line_magic('matplotlib', 'notebook')

dataset_dir = 'Segmentation_Dataset'
images_dir = os.path.join(dataset_dir, 'training')
training_dir = os.path.join(images_dir, 'images')
masks_dir = os.path.join(images_dir, 'masks')
test_dir = os.path.join(dataset_dir, 'test/images/img')

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    print(e)
    
# ------------------------------------------------UTILITY FUNCTIONS
    
def rle_encode(img):
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def saveTestDict(test_dir, IMAGE_DIMENSION, treshold):
    img_filenames = next(os.walk(test_dir))[2]
    
    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(csv_fname, 'w') as f:
        f.write('ImageId,EncodedPixels,Width,Height\n')
        s=0
        t=str(len(img_filenames))
        for img_filename in img_filenames:
            key = img_filename[:-4]
            if s % 50 is 0:
                print("Testing the image with id: " + str(key) + "... " + str(s+1) + "/" + t)
            img = Image.open(os.path.join(test_dir, img_filename))
            img = img.resize((IMAGE_DIMENSION, IMAGE_DIMENSION))

            img_arr = np.expand_dims(np.array(img), 0)
            out_sigmoid = model.predict(x=img_arr / 255.)

            for i in range(len(out_sigmoid[0])):
                for j in range(len(out_sigmoid[0][i])):
                    if out_sigmoid[0][i][j][0] > treshold:
                        out_sigmoid[0][i][j][0] = 1
                    else:
                        out_sigmoid[0][i][j][0] = 0

            predicted_class = (np.reshape(out_sigmoid, (1, IMAGE_DIMENSION, IMAGE_DIMENSION)))[0]

            target = np.array(Image.open(os.path.join(test_dir, img_filename)).resize((IMAGE_DIMENSION, IMAGE_DIMENSION)))
            prediction_img = np.zeros([target.shape[0], target.shape[1], 1])
            prediction_img[np.where(predicted_class == 0)] = 0
            prediction_img[np.where(predicted_class == 1)] = 1
            
            f.write(key + ',' + str(rle_encode(prediction_img)) + ',' + str(IMAGE_DIMENSION) + ',' + str(IMAGE_DIMENSION) + '\n')
            s = s + 1     

# ------------------------------------------------PARAMETERS DEFINITIONS (FOR EACH OF THE 4 STEPS)

for xa in range(4):
    IMAGE_DIMENSION = 256
    
    cwd = os.getcwd()

    img_h = IMAGE_DIMENSION
    img_w = IMAGE_DIMENSION
    print("\n\n")

    if xa is 0:
        SEED = 0
        tf.random.set_seed(SEED)  
        random.seed = SEED
        lr = 2e-5
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        validation_split = 0.05
        bs = 64
        IOU_index = 0.15
        apply_data_augmentation = True
        print("We are applying data augmentation!")
        epochs = 1
        VALIDATION = True  
        med_droput = 0.2
    if xa is 1:
        SEED = 1
        tf.random.set_seed(SEED)  
        random.seed = SEED
        lr = 0.05
        optimizer = tf.keras.optimizers.Adadelta(learning_rate=lr)
        validation_split = 0.01
        bs = 32
        IOU_index = 0.15
        apply_data_augmentation = False
        print("We are not applying data augmentation!")
        epochs = 1
        VALIDATION = True  
        med_droput = 0.001
    if xa is 2:
        SEED = 2
        tf.random.set_seed(SEED)  
        random.seed = SEED
        lr = 0.1
        optimizer = tf.keras.optimizers.Adadelta(learning_rate=lr)
        validation_split = 0.4
        bs = 8
        IOU_index = 0.5
        apply_data_augmentation = True
        print("We are applying data augmentation!")
        epochs = 1
        VALIDATION = False
    if xa is 3:
        SEED = 3
        tf.random.set_seed(SEED)  
        random.seed = SEED
        lr = 0.1
        optimizer = tf.keras.optimizers.Adadelta(learning_rate=lr)
        validation_split = 0.05
        bs = 2
        IOU_index = 0.5
        apply_data_augmentation = False
        print("We are not applying data augmentation!")
        epochs = 1
        VALIDATION = True


    print("\n\nWE ARE USING THIS PARAMETERS: lr=" + str(lr) + ", optimzer=" + str(optimizer) + 
          ", validation_split=" + str(validation_split))
    print("batch_size=" + str(bs) + ", IOU_Index=" + str(IOU_index))
    
# ------------------------------------------------DATA AUGMENTATION AND DATASETS GENERATORS DEFINITIONS

    data_gen_args = dict(brightness_range=[0.7, 1.0],
                         shear_range=0.2,
                         zoom_range=[1.0, 1.6],
                         horizontal_flip=True,
                         vertical_flip=True,
                         fill_mode='reflect',
                         validation_split = validation_split,
                         cval=0,
                         rescale=1./255)

    img_height = IMAGE_DIMENSION
    img_width = IMAGE_DIMENSION
    num_channels = 3
    img_shape = (img_height, img_width, num_channels)
    num_classes = 1

    # ---------------------------DATASET

    if apply_data_augmentation:
        train_img_data_gen = ImageDataGenerator(**data_gen_args)
        train_mask_data_gen = ImageDataGenerator(**data_gen_args)
    else:
        train_img_data_gen = ImageDataGenerator(validation_split = validation_split, rescale=1./255)
        train_mask_data_gen = ImageDataGenerator(validation_split = validation_split, rescale=1./255)

    valid_img_data_gen = ImageDataGenerator(rescale=1./255)
    valid_mask_data_gen = ImageDataGenerator(rescale=1./255)


    train_img_gen = train_img_data_gen.flow_from_directory(training_dir, target_size=(img_h, img_w), batch_size=bs, 
                                                            class_mode=None, shuffle=True, interpolation='bicubic',
                                                            subset='training', seed=SEED)  
    train_mask_gen = train_mask_data_gen.flow_from_directory(masks_dir, color_mode='grayscale', target_size=(img_h, img_w),
                                                                batch_size=bs, class_mode=None, shuffle=True,
                                                                interpolation='bicubic', subset='training', seed=SEED)
    train_gen = zip(train_img_gen, train_mask_gen)

    valid_img_gen = train_img_data_gen.flow_from_directory(training_dir, target_size=(img_h, img_w), batch_size=bs, 
                                                            class_mode=None, shuffle=True, interpolation='bicubic',
                                                            subset='validation', seed=SEED)
    valid_mask_gen = train_mask_data_gen.flow_from_directory(masks_dir, color_mode='grayscale', target_size=(img_h, img_w),
                                                                batch_size=bs, class_mode=None, shuffle=True,
                                                                interpolation='bicubic', subset='validation', seed=SEED)
    valid_gen = zip(valid_img_gen, valid_mask_gen)

    def prepare_target(x_, y_):
        y_ = tf.cast(y_, tf.float32)
        return x_, y_

    train_dataset = tf.data.Dataset.from_generator(lambda: train_gen, output_types=(tf.float32, tf.float32),
                                                    output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 1]))
    train_dataset = train_dataset.map(prepare_target)
    train_dataset = train_dataset.repeat()

    valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, output_types=(tf.float32, tf.float32),
                                                    output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 1]))
    valid_dataset = valid_dataset.map(prepare_target)
    valid_dataset = valid_dataset.repeat()

    # ---------------------------MODEL DEFINITION

    # ---------------------------------------ENCODER
    inputs = tf.keras.layers.Input((IMAGE_DIMENSION, IMAGE_DIMENSION, 3))
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(inputs)
    c1 = tf.keras.layers.Dropout(0.05)(c1)
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(c1)
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(c1)
    c1 = tf.keras.layers.Dropout(med_droput)(c1)                                
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1) #--

    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(p1)
    c2 = tf.keras.layers.Dropout(med_droput)(c2)
    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(c2)
    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(c2)     
    c2 = tf.keras.layers.Dropout(med_droput)(c2)                           
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2) #--

    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(p2)
    c3 = tf.keras.layers.Dropout(med_droput)(c3)
    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(c3)
    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(c3)
    c3 = tf.keras.layers.Dropout(med_droput)(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3) #--

    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(p3)
    c4 = tf.keras.layers.Dropout(med_droput)(c4)
    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(c4)
    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(c4)
    c4 = tf.keras.layers.Dropout(med_droput)(c4)
    p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4) #--

    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(p4)
    c5 = tf.keras.layers.Dropout(med_droput)(c5)
    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(c5) #--
    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(c5)
    c5 = tf.keras.layers.Dropout(med_droput)(c5)

    # ----------------------------------------------------------Plateu

    c5 = tf.keras.layers.Conv2D(512, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(c5)
    c5 = tf.keras.layers.Dropout(0.1)(c5)
    c5 = tf.keras.layers.Conv2D(512, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(c5)
    c5 = tf.keras.layers.Dropout(0.1)(c5)
    c5 = tf.keras.layers.Conv2D(512, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(c5)
    c5 = tf.keras.layers.Dropout(0.1)(c5)

    # ----------------------------------------------------------DECODER

    u6 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5) #--
    u6 = tf.keras.layers.concatenate([u6, c4])
    c6 = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(u6)
    c6 = tf.keras.layers.Dropout(med_droput)(c6)
    c6 = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(c6)
    c6 = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(c6)
    c6 = tf.keras.layers.Dropout(med_droput)(c6)                            

    u7 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6) #--
    u7 = tf.keras.layers.concatenate([u7, c3])
    c7 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(u7)
    c7 = tf.keras.layers.Dropout(med_droput)(c7)
    c7 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(c7)
    c7 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(c7)
    c7 = tf.keras.layers.Dropout(med_droput)(c7)

    u8 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c7) #--
    u8 = tf.keras.layers.concatenate([u8, c2])
    c8 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(u8)
    c8 = tf.keras.layers.Dropout(med_droput)(c8)
    c8 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(c8)
    c8 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(c8)
    c8 = tf.keras.layers.Dropout(med_droput)(c8)

    u9 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c8) #--
    u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
    c9 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(u9)
    c9 = tf.keras.layers.BatchNormalization()(c9)
    c9 = tf.keras.layers.Dropout(0.1)(c9)
    c9 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(c9)
    c9 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(c9)
    c9 = tf.keras.layers.Dropout(0.1)(c9)
    c9 = tf.keras.layers.Conv2D(4, (3, 3), activation=tf.keras.activations.elu,
                                padding='same')(c9)

    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9) #OUTPUT

    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    
    
# ------------------------------------------------MODEL COMPILATION (DEFINITION OF LOSS AND METRIC)

    def my_IoU(y_true, y_pred):
        y_pred = tf.cast(y_pred > IOU_index, tf.float32)
        intersection = tf.reduce_sum(y_true * y_pred)
        union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
        return intersection / union

    def jaccard_distance_loss(y_true, y_pred, smooth=100):
        intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
        sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
        jac = (intersection + smooth) / (sum_ - intersection + smooth)
        loss = (1 - jac) * smooth
        return loss

    model.compile(optimizer=optimizer, loss=jaccard_distance_loss, metrics=[my_IoU])
    #model.summary()

    LOAD_FROM_FILE = True
    try:
        if LOAD_FROM_FILE:
            if os.path.exists('my_model_weights.h5'):
                print("WEIGHTS ARE LOADED!!")
                model.load_weights('my_model_weights.h5')
    except Exception as e:
        print("WEIGHTS NOT LOADED!!")
        print(e)

    TRAIN = True
    if TRAIN:
        #reduceLR = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', factor=0.1, patience=4, mode='auto')
        #early = tf.keras.callbacks.EarlyStopping(monitor='accuracy', min_delta=1e-4, patience=7, mode='auto')

        if VALIDATION:
            model.fit(x=train_dataset, epochs=epochs, steps_per_epoch=len(train_img_gen), 
                      validation_data=valid_dataset, validation_steps=len(valid_img_gen))
        else:
            model.fit(x=train_dataset, epochs=epochs, steps_per_epoch=len(train_img_gen))
        

            

    SAVE = True
    if SAVE:
        model.save_weights('my_model_weights2.h5')
print()
print("--------------")
TEST = True
if TEST:
    saveTestDict(test_dir, IMAGE_DIMENSION, 0.28)

Using TensorFlow backend.





We are applying data augmentation!


WE ARE USING THIS PARAMETERS: lr=2e-05, optimzer=<tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x000001F748DEC608>, validation_split=0.05
batch_size=64, IOU_Index=0.15
Found 7265 images belonging to 1 classes.
Found 7265 images belonging to 1 classes.
Found 382 images belonging to 1 classes.
Found 382 images belonging to 1 classes.
Train for 114 steps, validate for 6 steps
  1/114 [..............................] - ETA: 2:58

KeyboardInterrupt: 

## Using transfer learning
Here instead we tried to use transfer learning in order to have final better performance from the model.
***
Also this time the final approach consists in 4 steps where the different parameters are changed. Further we used, for the downsampling part the **VGG** network, for the upsampling one the **pix2pix** network.

In [2]:
import tensorflow as tf

!pip install -q git+https://github.com/tensorflow/examples.git
from tensorflow_examples.models.pix2pix import pix2pix

from IPython.display import clear_output
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import FileLink, FileLinks

InteractiveShell.ast_node_interactivity = "all"

import os, sys, random, time, scipy.misc
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from PIL import Image

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
from keras.layers import BatchNormalization as BatchNorm
from keras.models import Model, load_model
from keras.layers import Input, Reshape, Activation
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from keras import layers

get_ipython().run_line_magic('matplotlib', 'notebook')

dataset_dir = 'Segmentation_Dataset'
images_dir = os.path.join(dataset_dir, 'training')
training_dir = os.path.join(images_dir, 'images')
masks_dir = os.path.join(images_dir, 'masks')
test_dir = os.path.join(dataset_dir, 'test/images/img')

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    print(e)
    
def rle_encode(img):
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def saveTestDict(test_dir, IMAGE_DIMENSION, treshold):
    img_filenames = next(os.walk(test_dir))[2]
    
    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(csv_fname, 'w') as f:

        f.write('ImageId,EncodedPixels,Width,Height\n')
        s=0
        t=str(len(img_filenames))
        for img_filename in img_filenames:
            key = img_filename[:-4]
            if s % 50 is 0:
                print("Testing the image with id: " + str(key) + "... " + str(s+1) + "/" + t)
            img = Image.open(os.path.join(test_dir, img_filename))
            img = img.resize((IMAGE_DIMENSION, IMAGE_DIMENSION))

            img_arr = np.expand_dims(np.array(img), 0)
            out_sigmoid = model.predict(x=img_arr / 255.)

            for i in range(len(out_sigmoid[0])):
                for j in range(len(out_sigmoid[0][i])):
                    if out_sigmoid[0][i][j][0] > treshold:
                        out_sigmoid[0][i][j][0] = 1
                    else:
                        out_sigmoid[0][i][j][0] = 0

            predicted_class = (np.reshape(out_sigmoid, (1, IMAGE_DIMENSION, IMAGE_DIMENSION)))[0]

            target = np.array(Image.open(os.path.join(test_dir, img_filename)).resize((IMAGE_DIMENSION, IMAGE_DIMENSION)))
            prediction_img = np.zeros([target.shape[0], target.shape[1], 1])
            prediction_img[np.where(predicted_class == 0)] = 0
            prediction_img[np.where(predicted_class == 1)] = 1
            
            f.write(key + ',' + str(rle_encode(prediction_img)) + ',' + str(IMAGE_DIMENSION) + ',' + str(IMAGE_DIMENSION) + '\n')
            s = s + 1     

# ---------------------------PARAMETERS

for xa in range(4):
    IMAGE_DIMENSION = 256
    
    cwd = os.getcwd()

    img_h = IMAGE_DIMENSION
    img_w = IMAGE_DIMENSION
    print("\n\n")

    if xa is 0:
        SEED = 0
        tf.random.set_seed(SEED)  
        random.seed = SEED
        lr = 1.0
        optimizer = tf.keras.optimizers.Adadelta(learning_rate=lr)
        validation_split = 0.1
        bs = 16
        IOU_index = 0.5
        apply_data_augmentation = True
        print("We are applying data augmentation!")
        epochs = 20
        VALIDATION = True  
        med_droput = 0.1
    if xa is 1:
        SEED = 1
        tf.random.set_seed(SEED)  
        random.seed = SEED
        lr = 1.0
        optimizer = tf.keras.optimizers.Adadelta(learning_rate=lr)
        validation_split = 0.1
        bs = 16
        IOU_index = 0.5
        apply_data_augmentation = False
        print("We are not applying data augmentation!")
        epochs = 20
        VALIDATION = True  
        med_droput = 0.1
    if xa is 2:
        SEED = 2
        tf.random.set_seed(SEED)  
        random.seed = SEED
        lr = 0.5
        optimizer = tf.keras.optimizers.Adadelta(learning_rate=lr)
        validation_split = 0.1
        bs = 16
        IOU_index = 0.5
        apply_data_augmentation = True
        print("We are applying data augmentation!")
        epochs = 20
        VALIDATION = True  
        med_droput = 0.1
    if xa is 3:
        SEED = 3
        tf.random.set_seed(SEED)  
        random.seed = SEED
        lr = 0.5
        optimizer = tf.keras.optimizers.Adadelta(learning_rate=lr)
        validation_split = 0.1
        bs = 16
        IOU_index = 0.5
        apply_data_augmentation = False
        print("We are not applying data augmentation!")
        epochs = 20
        VALIDATION = True  
        med_droput = 0.1

    print("\n\nWE ARE USING THIS PARAMETERS: lr=" + str(lr) + ", optimzer=" + 
          str(optimizer) + ", validation_split=" + str(validation_split))
    print("batch_size=" + str(bs) + ", IOU_Index=" + str(IOU_index))
    
    data_gen_args = dict(brightness_range=[0.7, 1.0],
                         shear_range=0.2,
                         zoom_range=[1.0, 1.6],
                         horizontal_flip=True,
                         vertical_flip=True,
                         fill_mode='reflect',
                         validation_split = validation_split,
                         cval=0,
                         rescale=1./255)

    img_height = IMAGE_DIMENSION
    img_width = IMAGE_DIMENSION
    num_channels = 3
    img_shape = (img_height, img_width, num_channels)
    num_classes = 1

    # ---------------------------DATASET

    
    
    if apply_data_augmentation:
        train_img_data_gen = ImageDataGenerator(**data_gen_args)
        train_mask_data_gen = ImageDataGenerator(**data_gen_args)
    else:
        train_img_data_gen = ImageDataGenerator(validation_split = validation_split, rescale=1./255)
        train_mask_data_gen = ImageDataGenerator(validation_split = validation_split, rescale=1./255)

    valid_img_data_gen = ImageDataGenerator(rescale=1./255)
    valid_mask_data_gen = ImageDataGenerator(rescale=1./255)


    train_img_gen = train_img_data_gen.flow_from_directory(training_dir, target_size=(img_h, img_w), batch_size=bs, 
                                                            class_mode=None, shuffle=True, interpolation='bicubic',
                                                            subset='training', seed=SEED)  
    train_mask_gen = train_mask_data_gen.flow_from_directory(masks_dir, color_mode='grayscale', target_size=(img_h, img_w),
                                                                batch_size=bs, class_mode=None, shuffle=True,
                                                                interpolation='bicubic', subset='training', seed=SEED)
    train_gen = zip(train_img_gen, train_mask_gen)

    valid_img_gen = train_img_data_gen.flow_from_directory(training_dir, target_size=(img_h, img_w), batch_size=bs, 
                                                            class_mode=None, shuffle=True, interpolation='bicubic',
                                                            subset='validation', seed=SEED)
    valid_mask_gen = train_mask_data_gen.flow_from_directory(masks_dir, color_mode='grayscale', target_size=(img_h, img_w),
                                                                batch_size=bs, class_mode=None, shuffle=True,
                                                                interpolation='bicubic', subset='validation', seed=SEED)
    valid_gen = zip(valid_img_gen, valid_mask_gen)

    def prepare_target(x_, y_):
        y_ = tf.cast(y_, tf.float32)
        return x_, y_


    train_dataset = tf.data.Dataset.from_generator(lambda: train_gen, output_types=(tf.float32, tf.float32),
                                                    output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 1]))
    train_dataset = train_dataset.map(prepare_target)
    train_dataset = train_dataset.repeat()

    valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, output_types=(tf.float32, tf.float32),
                                                    output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 1]))
    valid_dataset = valid_dataset.map(prepare_target)
    valid_dataset = valid_dataset.repeat()

    # ---------------------------MODEL

    base_model = tf.keras.applications.VGG16(input_shape=[IMAGE_DIMENSION, IMAGE_DIMENSION, 3], include_top=False)
    

    # Use the activations of these layers
    layer_names = [
                   'block1_conv2',
                   'block2_conv2',
                   'block3_conv3',
                   'block4_conv3',
                   'block5_conv3'
                   ]
    layers = [base_model.get_layer(name).output for name in layer_names]

    # Create the feature extraction model
    down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

    down_stack.trainable = False

    up_stack = [
        pix2pix.upsample(512, 3),  # 4x4 -> 8x8
        pix2pix.upsample(256, 3),  # 8x8 -> 16x16
        pix2pix.upsample(128, 3)   # 32x32 -> 64x64
    ]

    
    def unet_model(output_channels):
        # This is the last layer of the model
        last = tf.keras.layers.Conv2DTranspose(
            output_channels, 3, strides=2,
            padding='same', activation='sigmoid')  #64x64 -> 128x128

        inputs = tf.keras.layers.Input(shape=[IMAGE_DIMENSION, IMAGE_DIMENSION, 3])
        x = inputs

        # Downsampling through the model
        skips = down_stack(x)
        x = skips[-1]
        skips = reversed(skips[:-1])

        # Upsampling and establishing the skip connections
        for up, skip in zip(up_stack, skips):
            x = up(x)
            concat = tf.keras.layers.Concatenate()
            x = concat([x, skip])

        #x = tf.keras.layers.Dropout(med_droput)
        x = last(x)

        return tf.keras.Model(inputs=inputs, outputs=x)

    model = unet_model(1)

    #model.summary()
    #sys.exit()
    # ---------------------------------------ENCODER
    
    

    def my_IoU(y_true, y_pred):
        y_pred = tf.cast(y_pred > IOU_index, tf.float32)
        intersection = tf.reduce_sum(y_true * y_pred)
        union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
        return intersection / union

    def jaccard_distance_loss(y_true, y_pred, smooth=100):
        intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
        sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
        jac = (intersection + smooth) / (sum_ - intersection + smooth)
        loss = (1 - jac) * smooth
        return loss

    def iou_coef(y_true, y_pred, smooth=1):
        intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
        union = K.sum(y_true,-1) + K.sum(y_pred,-1) - intersection
        a = intersection + smooth
        b = union + smooth
        return a / b

    def iou_coef_loss(y_true, y_pred):
        return 1-iou_coef(y_true, y_pred)

    
    if xa is 0:
        model.compile(optimizer=optimizer, loss=jaccard_distance_loss, metrics=[iou_coef])
    if xa is 1:
        model.compile(optimizer=optimizer, loss=jaccard_distance_loss, metrics=[iou_coef])
    if xa is 2:
        model.compile(optimizer=optimizer, loss=jaccard_distance_loss, metrics=[my_IoU])
    if xa is 3: 
        model.compile(optimizer=optimizer, loss=jaccard_distance_loss, metrics=[my_IoU])

    LOAD_FROM_FILE = True
    try:
        if LOAD_FROM_FILE:
            if os.path.exists('my_model_weights2.h5'):
                print("WEIGHTS FROM KAGGLE/WORKING ARE LOADED!!")
                model.load_weights('my_model_weights2.h5')
            else:
                if os.path.exists('/kaggle/input/lastws/my_model_weights.h5'):
                    print("WEIGHTS FROM input database ARE LOADED!!")
                    model.load_weights('/kaggle/input/lastws/my_model_weights.h5')
    except Exception as e:
        print("WEIGHTS NOT LOADED!!")
        print(e)

    TRAIN = True
    if TRAIN:
        #reduceLR = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', factor=0.1, patience=4, mode='auto')
        #early = tf.keras.callbacks.EarlyStopping(monitor='accuracy', min_delta=1e-4, patience=7, mode='auto')

        if VALIDATION:
            model.fit(x=train_dataset, epochs=epochs, steps_per_epoch=len(train_img_gen), 
                      validation_data=valid_dataset, validation_steps=len(valid_img_gen))
        else:
            model.fit(x=train_dataset, epochs=epochs, steps_per_epoch=len(train_img_gen))
        

            

    SAVE = True
    if SAVE:
        model.save_weights('my_model_weights2.h5')
print()
print("--------------")
TEST = False
if TEST:
    saveTestDict(test_dir, IMAGE_DIMENSION, 0.28)




We are applying data augmentation!


WE ARE USING THIS PARAMETERS: lr=1.0, optimzer=<tensorflow.python.keras.optimizer_v2.adadelta.Adadelta object at 0x000001F748E02388>, validation_split=0.1
batch_size=16, IOU_Index=0.5
Found 6883 images belonging to 1 classes.
Found 6883 images belonging to 1 classes.
Found 764 images belonging to 1 classes.
Found 764 images belonging to 1 classes.
58892288/58889256 [==============================] - 13s 0us/step
Train for 431 steps, validate for 48 steps
Epoch 1/20
  1/431 [..............................] - ETA: 59:02

KeyboardInterrupt: 